思路：

拼接所有能合并的特征
能拆就拆
然后全部one-hot

对于AppID，先把一个用户的AppID连接在一起，然后使用tf-idf处理，得到App特征

上下两个合起来，裸跑LogisticRegression

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import numpy as np
import pickle
import math
import cPickle
import xgboost as xgb

In [2]:
#评分函数
import scipy as sp
def logloss(act, pred):
  epsilon = 1e-15
  pred = sp.maximum(epsilon, pred)
  pred = sp.minimum(1-epsilon, pred)
  ll = -sp.mean(act*sp.log(pred) + sp.subtract(1,act)*sp.log(1-pred))
  return ll

In [8]:
train = pd.read_csv('./pre/train.csv')
test = pd.read_csv('./pre/test.csv')

In [25]:
# statead = pd.read_csv('./statead.csv')
# statead.head()

,creativeID,brand,successclick,successconversion
0,4079,191,2890.0,10
1,4565,56,449000.0,11622
2,3170,1400,387.0,5
3,6566,1400,224.0,1
4,5187,56,141000.0,2277


In [9]:
#时间离散化
train['clickTime_day'] = train['clickTime'].map(lambda x:int(x/10000))
train['clickTime_hour'] = train['clickTime'].map(lambda x:int(x/100%100))
train['clickTime_minute'] = train['clickTime'].map(lambda x:int(x%100))

In [10]:
train.groupby(['clickTime_day'])['label'].value_counts()

clickTime_day  label
17             0        287089
               1          7464
18             0        155890
               1          4101
19             0        100872
               1          3286
20             0        201406
               1          5056
21             0        301475
               1          7121
22             0        318550
               1          7371
23             0        280826
               1          7607
24             0        277860
               1          7382
25             0        259457
               1          7376
26             0        290089
               1          7736
27             0        289065
               1          7649
28             0        271291
               1          7343
29             0        294307
               1          7462
30             0        328089
               1          6308
Name: label, dtype: int64

In [11]:
# 将第28天作为验证集  （集第一次更新）
# proof = train[train.clickTime_day==28]
train = train[(train.clickTime_day>=17 ) & (train.clickTime_day<= 28)]

In [12]:
print test.shape,train.shape

(338489, 8) (3113362, 11)


In [13]:
#时间离散化
test['clickTime_day'] = test['clickTime'].map(lambda x:int(x/10000))
test['clickTime_hour'] = test['clickTime'].map(lambda x:int(x/100%100))
test['clickTime_minute'] = test['clickTime'].map(lambda x:int(x%100))

In [30]:
test.groupby(['clickTime_hour'])['label'].value_counts()

clickTime_hour  label
0               -1        7545
1               -1        4785
2               -1        3159
3               -1        2478
4               -1        2643
5               -1        3329
6               -1        6014
7               -1       10554
8               -1       13330
9               -1       15433
10              -1       17187
11              -1       16355
12              -1       17028
13              -1       17776
14              -1       19110
15              -1       19717
16              -1       18122
17              -1       18839
18              -1       20482
19              -1       23049
20              -1       22770
21              -1       24124
22              -1       20210
23              -1       14450
Name: label, dtype: int64

In [14]:
#position直接加上去，LogisticRegression Logistic回归
#的训练得分 0.120106201117，可见position特征用处不大
position = pd.read_csv('./pre/position.csv')
# train = pd.merge(train,statead,position,on='positionID',how='left')
# test = pd.merge(test,statead,position,on='positionID',how='left')
position.head()

,positionID,sitesetID,positionType
0,2150,1,0
1,2579,1,0
2,3322,1,0
3,5726,1,0
4,4522,2,0


In [15]:
# train = pd.merge(train,statead,on='creativeID',how='left')
# test = pd.merge(test,statead,on='creativeID',how='left')

NameError: name 'statead' is not defined

In [23]:
# statead.head()

,creativenessID,brand,successclick,successconversion
0,4079,191,2890.0,10
1,4565,56,449000.0,11622
2,3170,1400,387.0,5
3,6566,1400,224.0,1
4,5187,56,141000.0,2277


In [16]:
#numberical feature数字特征，feature_name总特征，categorical_feature 分类特征
#我们去掉label，convertiontime跑一次
feature_name = [a for a in train.columns if a not in ['label','conversionTime']]
categorical_feature = ['creativeID','userID','positionID','connectionType','telecomsOperator']

In [17]:
#去掉除label，convertiontime的第二次数据集
train_label = train['label']
train = train[feature_name]
test_label = test['label']
test = test[feature_name]

In [18]:
#添加appID特征（tfidf）
user_installedapps = pd.read_csv('./pre/user_installedapps.csv')
user_installedapps_count = user_installedapps.groupby('userID').agg(len).reset_index()#计数特征


In [19]:
user_installedapps.head()

,userID,appID
0,1,357
1,1,360
2,1,362
3,1,365
4,1,375


In [20]:
user_installedapps_count.columns = ['userID','user_appID_count']
#2798058	app360 app361 app362 app375 app480 app481 app4  相当于app+value
user_installedapps = user_installedapps.groupby('userID').agg(lambda x:' '.join(['app'+str(s) for s in x.values])).reset_index()



In [21]:
user_id_all = pd.concat([train.userID,test.userID],axis=0)
user_id_all = pd.DataFrame(user_id_all,columns=['userID'])
user_id_all.head()



,userID
0,2798058
1,463234
2,1857485
3,2038823
4,2015141


In [22]:
#不同用户的先提取出来
user_installedapps = pd.merge(user_id_all.drop_duplicates(),user_installedapps,on='userID',how='left')
user_installedapps = user_installedapps.fillna('Missing')
#至此，user_installedapps处理完毕

In [23]:

tfv = TfidfVectorizer()
tfv.fit(user_installedapps.appID)



TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [24]:
#按照顺序转化为tfidf特征
user_installedapps = pd.merge(user_id_all,user_installedapps,on='userID',how='left')
user_installedapps = user_installedapps.fillna('Missing')
user_installedapps_tfv = tfv.transform(user_installedapps.appID)

In [22]:
#保险起见，爱你，就储存吧
user_installedapps.to_csv('./pre/user-app.csv',index=None)

In [26]:
def featureManipulation(dtfm, colList, func):
    '''依次处理某一dataframe内__所有__col的__所有__零值'''
    for col in colList:
        pr_col = func(dtfm, col)
        for row in pr_col.iterrows():
            zeroSample = dtfm[col][(dtfm[col] == 0)]
            replace = row[0]
            num = row[1][col].astype(int)
            if num > len(zeroSample):
                print(replace)
                num = len(zeroSample)
            if num <= 0:
                continue
            smpl = zeroSample.sample(num)
            smpl = smpl.replace(0, replace)
            dtfm[col].update(smpl)
    print(dtfm)

In [27]:
# 这里是对user的例子
user = pd.read_csv('./pre/user.csv')
user.head()
def sln(dtfm, col):
    dtfm_col = dtfm[dtfm[col] > 0]
    pr_col = dtfm_col[col].value_counts()/len(dtfm_col[col])
    pr_col *= len(dtfm[col][(dtfm[col] == 0)])
    pr_col = pr_col.apply(np.round)
    pr_col = pr_col.to_frame()
    return pr_col
featureManipulation(user, ['age','gender','education','hometown','residence'], sln)

3207
3203
3206
          userID  age  gender  education  marriageStatus  haveBaby  hometown  \
0              1   42       1          1               2         0       512   
1              2   18       1          5               1         0      1403   
2              3   11       2          4               0         0      1006   
3              4   21       2          5               3         0       607   
4              5   22       2          3               0         0       101   
5              6   20       2          5               0         0       301   
6              7   17       1          5               0         0       313   
7              8   21       1          2               3         1      1607   
8              9   38       2          2               2         0      2203   
9             10   13       2          1               0         0       311   
10            11   17       2          3               2         1      2405   
11            12   15    

In [13]:
user.isnull().values.any()

False

In [28]:
user['hometown_city'] = user['hometown']%100
user['hometown_province'] = (user['hometown']/100).astype('int')
user['residence_city'] = user['residence']%100
user['residence_province'] = (user['residence']/100).astype('int')

In [29]:
ad = pd.read_csv('./pre/ad.csv')
ad.head()

,creativeID,adID,camgaignID,advertiserID,appID,appPlatform
0,4079,2318,147,80,14,2
1,4565,3593,632,3,465,1
2,3170,1593,205,54,389,1
3,6566,2390,205,54,389,1
4,5187,411,564,3,465,1


In [30]:
#合并特征
train = pd.merge(train,user_installedapps_count,on='userID',how='left')
train = pd.merge(train,user,on='userID',how='left')
train = pd.merge(train,ad,on='creativeID',how='left')



In [31]:
#验证集合并特征
test = pd.merge(test,user_installedapps_count,on='userID',how='left')
test = pd.merge(test,user,on='userID',how='left')
test = pd.merge(test,ad,on='creativeID',how='left') 

In [33]:
train.shape

(3113362, 26)

In [34]:
#保险起见，爱你，就储存吧
train.to_csv('./pre/train17-28.csv',index=None)

In [51]:
train = pd.read_csv('./pre/train17-28.csv')
train

,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator,clickTime_day,clickTime_hour,clickTime_minute,user_appID_count,...,residence,hometown_city,hometown_province,residence_city,residence_province,adID,camgaignID,advertiserID,appID,appPlatform
0,170000,3089,2798058,293,1,1,17,0,0,44.0,...,1301,7,2,1,13,1321,83,10,434,1
1,170000,1259,463234,6161,1,2,17,0,0,NaN,...,213,6,6,13,2,1535,685,80,14,2
2,170000,4465,1857485,7434,4,1,17,0,0,NaN,...,1502,5,8,2,15,147,460,3,465,1
3,170000,1004,2038823,977,1,1,17,0,0,69.0,...,1001,6,5,1,10,411,564,3,465,1
4,170000,1887,2015141,3688,1,1,17,0,0,NaN,...,1001,1,10,1,10,369,144,84,360,1
5,170000,3293,1177829,3347,1,1,17,0,0,NaN,...,107,7,1,7,1,2891,685,80,14,2
6,170000,4793,1257450,1876,1,2,17,0,0,28.0,...,1104,1,13,4,11,1225,484,80,14,2
7,170000,1456,764204,6086,1,3,17,0,0,78.0,...,308,8,3,8,3,3379,411,3,465,1
8,170000,4465,1438585,2426,2,3,17,0,0,NaN,...,601,3,8,1,6,147,460,3,465,1
9,170000,985,1750317,5414,1,3,17,0,0,88.0,...,2102,2,21,2,21,86,503,3,465,1


In [35]:
test.shape

(338489, 26)

In [36]:
#保险起见，爱你，就储存吧
test.to_csv('./pre/test17-28.csv',index=None)

In [49]:
test = pd.read_csv('./pre/test17-28.csv')
test

,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator,clickTime_day,clickTime_hour,clickTime_minute,user_appID_count,...,residence,hometown_city,hometown_province,residence_city,residence_province,adID,camgaignID,advertiserID,appID,appPlatform
0,310000,3745,1164848,3451,1,3,31,0,0,34.0,...,605,5,6,5,6,1166,430,80,14,2
1,310000,2284,2127247,1613,1,3,31,0,0,NaN,...,2301,5,22,1,23,1388,325,80,14,2
2,310000,1456,2769125,5510,2,1,31,0,0,74.0,...,1502,3,10,2,15,3379,411,3,465,1
3,310000,4565,9762,4113,2,3,31,0,0,NaN,...,2407,5,24,7,24,3593,632,3,465,1
4,310000,49,2513636,3615,1,3,31,0,0,NaN,...,307,7,3,7,3,1469,535,80,14,2
5,310000,3824,488035,3821,1,1,31,0,0,NaN,...,209,5,24,9,2,2792,252,79,391,1
6,310000,863,225561,4188,1,3,31,0,0,NaN,...,111,5,6,11,1,2238,637,3,465,1
7,310000,4469,1144620,4113,1,1,31,0,0,90.0,...,1408,1,5,8,14,2853,138,79,391,1
8,310000,3745,1113275,3347,1,2,31,0,0,66.0,...,504,10,5,4,5,1166,430,80,14,2
9,310000,4565,1361729,2426,2,2,31,0,0,92.0,...,1201,1,12,1,12,3593,632,3,465,1


In [6]:
#去掉除label，convertiontime的第二次数据集
train_label = train['label']
test_label = test['label']

KeyError: 'label'

In [47]:
train.dtypes

clickTime             int64
creativeID            int64
userID                int64
positionID            int64
connectionType        int64
telecomsOperator      int64
clickTime_day         int64
clickTime_hour        int64
clickTime_minute      int64
age                   int64
gender                int64
education             int64
marriageStatus        int64
haveBaby              int64
hometown              int64
residence             int64
hometown_city         int64
hometown_province     int32
residence_city        int64
residence_province    int32
adID                  int64
camgaignID            int64
advertiserID          int64
appID                 int64
appPlatform           int64
dtype: object

In [91]:
train = train.fillna(0)
test = test.fillna(0)
train.dtypes

clickTime               int64
creativeID              int64
userID                  int64
positionID              int64
connectionType          int64
telecomsOperator        int64
clickTime_day           int64
clickTime_hour          int64
clickTime_minute        int64
sitesetID               int64
positionType            int64
user_appID_count      float64
age                     int64
gender                  int64
education               int64
marriageStatus          int64
haveBaby                int64
hometown                int64
residence               int64
hometown_city           int64
hometown_province       int32
residence_city          int64
residence_province      int32
adID                    int64
camgaignID              int64
advertiserID            int64
appID                   int64
appPlatform             int64
dtype: object

In [52]:
train_user_appID_count =  train[['user_appID_count']]
test_user_appID_count =  test[['user_appID_count']]
del train['user_appID_count'],test['user_appID_count']

In [93]:
oneEnc = OneHotEncoder()
data_one = pd.concat([train,test])
data_one = oneEnc.fit_transform(data_one)
train_one = data_one[:train.shape[0]]
test_one = data_one[train.shape[0]:]

In [94]:
print train_one.shape
print user_installedapps_tfv[:train.shape[0]].shape
print train_user_appID_count.shape
print train.shape

(861372, 987038)
(861372, 111975)
(861372, 1)
(861372, 27)


In [95]:
train_user_appID_count.values

array([[ 64.],
       [  0.],
       [ 67.],
       ..., 
       [ 64.],
       [  0.],
       [ 92.]])

In [100]:
train = hstack([train_one,user_installedapps_tfv[:train.shape[0]]])
test = hstack([test_one,user_installedapps_tfv[train.shape[0]:]])

In [13]:
# #输出训练集和测试集
# with open('train.pkl','w') as f:
#     pickle.dump(train,f)
# with open('test.pkl','w') as f:
#     pickle.dump(test,f)
#读取训练集和测试集
with open('train.pkl','rb') as f:
    train = cPickle.load(f)
# with open('test.pkl','rb') as f:
#     test = cPickle.load(f)

In [15]:
 print(test.predict(X[0:1]))

AttributeError: predict not found

In [97]:
# from sklearn.linear_model import LogisticRegression
# print 'LogisticRegression Logistic回归'
# lr = LogisticRegression(n_jobs=-1,random_state=2017)
# lr.fit(train,train_label)
# pred = lr.predict_proba(train)[:,1]
# print '训练得分',logloss(train_label,pred)
# # pred = lr.predict_proba(test)[:,1]
# # print '验证得分',logloss(test_label,pred)

LogisticRegression Logistic回归
训练得分 0.0797782026628


In [53]:
#模型参数设置
xlf = xgb.XGBRegressor(max_depth=5, 
                        learning_rate=0.01, 
                        n_estimators=2000, 
                        silent=True, 
                        objective='reg:linear', 
                        n_jobs=-1, 
                        gamma=0.1,
                        min_child_weight=1.1, 
                        max_delta_step=5, 
                        subsample=0.7, 
                        colsample_bytree=0.7, 
                        colsample_bylevel=0.7, 
                        reg_alpha=0, 
                        reg_lambda=10, 
                        scale_pos_weight=1, 
                        random_state=0, 
                        missing=None)

xlf.fit(train, train_label, eval_metric='rmse', eval_set = [(test,test_label )],verbose = True, early_stopping_rounds=200)


# pred = lr.predict_proba(test)[:,1]
# print '验证得分',logloss(test_label,pred)

[0]	validation_0-rmse:1.49526
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:1.49061
[2]	validation_0-rmse:1.48599
[3]	validation_0-rmse:1.48146
[4]	validation_0-rmse:1.47694
[5]	validation_0-rmse:1.4724
[6]	validation_0-rmse:1.46797
[7]	validation_0-rmse:1.46354
[8]	validation_0-rmse:1.45916
[9]	validation_0-rmse:1.45483
[10]	validation_0-rmse:1.45062
[11]	validation_0-rmse:1.44642
[12]	validation_0-rmse:1.44227
[13]	validation_0-rmse:1.43838
[14]	validation_0-rmse:1.4343
[15]	validation_0-rmse:1.43024
[16]	validation_0-rmse:1.4262
[17]	validation_0-rmse:1.42223
[18]	validation_0-rmse:1.41832
[19]	validation_0-rmse:1.41444
[20]	validation_0-rmse:1.41057
[21]	validation_0-rmse:1.40688
[22]	validation_0-rmse:1.40307
[23]	validation_0-rmse:1.39935
[24]	validation_0-rmse:1.39565
[25]	validation_0-rmse:1.39196
[26]	validation_0-rmse:1.38836
[27]	validation_0-rmse:1.3848
[28]	validation_0-rmse:1.38124
[29]	validation_0-rmse:1.37792
[30]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=-1, nthread=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.85)

In [54]:
pred = xlf.predict(train)
print '训练得分',logloss(train_label,pred)

训练得分 0.100489348173


In [55]:
results = xlf.predict(test)
test['prob'] = results

In [101]:
# results = lr.predict_proba(test)[:,1]
# print '验证得分',logloss(test_label,pred)

验证得分

ValueError: operands could not be broadcast together with shapes (338489,) (861372,) 

In [56]:
#输出结果
# a = pd.DataFrame({'instanceID':pd.read_csv('./pre/test.csv')['instanceID'],'prob':pred})
#输出
test1 = pd.read_csv('./pre/test.csv')
test1['prob'] = results
test1= test1[['instanceID','prob']]
test1.to_csv('./pre/submission.csv',index=None)
submission =  pd.read_csv('./pre/submission.csv') 

In [ ]:
def get_duplicated_feature():
    #重复数据是本题的一个特点
    #部分label为1的特征是重复的而且位置为第一个
    
    train = pd.read_csv('../input/train.csv')
    train.drop('conversionTime',axis=1,inplace=True)
    train.drop('label',axis=1,inplace=True)
    test = pd.read_csv('../input/test.csv')
    test.drop('instanceID',axis=1,inplace=True)
    test.drop('label',axis=1,inplace=True)
    
    is_duplicated = train.duplicated(keep=False).astype('int')
    is_duplicated_first = train.duplicated(keep='first').astype('int')
    is_duplicated_last = train.duplicated(keep='last').astype('int')
    train['is_duplicated'] = is_duplicated
    train['is_duplicated_first'] = is_duplicated_first
    train['is_duplicated_last'] = is_duplicated_last
    
    is_duplicated = test.duplicated(keep=False).astype('int')
    is_duplicated_first = test.duplicated(keep='first').astype('int')
    is_duplicated_last = test.duplicated(keep='last').astype('int')
    test['is_duplicated'] = is_duplicated
    test['is_duplicated_first'] = is_duplicated_first
    test['is_duplicated_last'] = is_duplicated_last
    
    return train[['is_duplicated','is_duplicated_first','is_duplicated_last']],test[['is_duplicated','is_duplicated_first','is_duplicated_last']]


In [57]:
submission

,instanceID,prob
0,1,0.006102
1,2,0.008251
2,3,0.030742
3,4,0.034045
4,5,0.021748
5,6,0.038723
6,7,0.039754
7,8,0.074027
8,9,0.008836
9,10,0.012768
